# Run one robot for one day

Run an experiment with a single robot exploring an environment during the course of one day.

The specification of the experiment is fully determined by the exp/run of type __1robot1day__. This exp/run will refer to other exp/runs to specify the environment, the robot policy, the estimator and the scoring function. 

The result of running this notebook is to run the experiment and save the results into the exp/run datadir in a file called "results.pickle". 

In [1]:
import sys
sys.path.append("..")
from exp_run_config import Config
Config.PROJECTNAME = "WaterBerryFarms"
import pathlib
from pprint import pprint
import gzip as compress
import copy
import pickle

from wbf_simulate import simulate_1day, save_simulation_results
from wbf_helper import get_geometry, create_wbfe, create_policy, create_estimator, create_score

from robot import Robot


In [2]:

# the experiment for the robot run
experiment = "1robot1day"
run = "rwp-ad-30"
# run = "fblm-ad-30"
# run = "fblm-gp-30" # this one with GP takes about 10 minutes to run (1 min office machine, 500 steps)

exp = Config().get_experiment(experiment, run)
# print(exp.experiment())
# print(exp.run())
# print(exp.subrun())
# pprint(exp)

resultsfile = pathlib.Path(exp.data_dir(), "results.pickle")
if resultsfile.exists():
    print(f"Results file already exists:\n{resultsfile}")
    print(f"Delete this file if re-running is desired.")
    raise Exception("Nothing to do.")

# the exp for the environment
exp_env = Config().get_experiment("environment", exp["exp_environment"])
pprint(exp_env)
# the exp for policy
exp_policy = Config().get_experiment("policy", exp["exp_policy"])
pprint(exp_policy)
# the exp for estimator
exp_estimator = Config().get_experiment("estimator", exp["exp_estimator"])
pprint(exp_estimator)
# the exp for the score
exp_score = Config().get_experiment("score", exp["exp_score"])
pprint(exp_score)

***ExpRun**: Loading pointer config file:
	C:\Users\lboloni\.config\WaterBerryFarms\mainsettings.yaml
***ExpRun**: Loading machine-specific config file:
	G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\WBF-config-LotziYoga.yaml
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\1robot1day\rwp-ad-30_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: 1robot1day/rwp-ad-30 successfully loaded
***ExpRun**: Experiment default config C:\Users\lboloni\Documents\Code\_Checkouts\WBF\WaterberryFarms\experiment_configs\environment\_environment.yaml was empty, ok.
***ExpRun**: No system dependent experiment file
	 G:\My Drive\LotziStudy\Code\PackageTracking\WaterBerryFarms\settings\experiment-config\LotziYoga\environment\miniberry-30_sysdep.yaml,
	 that is ok, proceeding.
***ExpRun**: Configuration for exp/run: environment/miniberry-30 successfully loaded

In [3]:

# not clear what the picture paths do here
picture_paths = []

results = {}

if exp_policy["policy-code"] == "-":
    generator = exp_policy["policy-code-generator"]        
    policy = eval(generator)(exp_policy, exp_env)
else:
    policy = create_policy(exp_policy, exp_env)
results["policy-code"] = policy
results["policy-name"] = results["policy-code"].name

results["estimator-CODE"] = create_estimator(exp_estimator, exp_env)
results["estimator-name"] = results["estimator-CODE"].name

results["score-code"] = create_score(exp_score, exp_env)
results["score-name"] = results["score-code"].name

results["velocity"] = exp["velocity"]
results["timesteps-per-day"] = exp["timesteps-per-day"]
results["time-start-environment"] = exp["time-start-environment"]
results["im_resolution"] = exp["im_resolution"]
results["results-basedir"] = exp["data_dir"]
results["action"] = "run-one-day"
results["typename"] = exp_env["typename"]
wbf, wbfe = create_wbfe(exp_env)
# move ahead to the starting point of the environment
wbfe.proceed(results["time-start-environment"])
results["wbf"] = wbf
results["wbfe"] = wbfe
results["days"] = 1
get_geometry(results["typename"], results)
# create the robot and set the policy
results["robot"] = Robot("Rob", 0, 0, 0, env=None, im=None)
results["robot"].assign_policy(results["policy-code"])
# 
# This is where we actually calling the simulation
#
simulate_1day(results)
#print(f"Saving results to: {resultsfile}")
#with compress.open(resultsfile, "wb") as f:
#    pickle.dump(results, f)
save_simulation_results(resultsfile, results)
exp.done()

INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 6
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-30\precalc_tylcv\env_value_00006.gz
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-30\precalc_tylcv\env_value_00006.gz done
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 6
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-30\precalc_ccr\env_value_00006.gz
INFO:root:Loading from gz C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\environment\miniberry-30\precalc_ccr\env_value_00006.gz done
INFO:root:Environment.proceed - calling the inner_proceed
INFO:root:PrecalculatedEnvironment at timestamp 6
INFO:root:Loading from gz C:\Use

loading the geometry and environment from saved data
loading done
Saving results to: C:\Users\lboloni\Documents\Code\_TempData\WBF\ExperimentData\1robot1day\rwp-ad-30\results.pickle
